# 3.3 モデルの実装

### 共通事前処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# 乱数の種
random_seed = 123

### 乳がん疾患データセット

[UCIデータセットサイト](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))

#### 分析データイメージ
<img src="https://www.researchgate.net/profile/Nick_Street/publication/2512520/figure/fig2/AS:279373199495169@1443619169198/Snakes-After-Convergence-to-Cell-Nucleus-Boundaries-These-contours-are-the-nal.png" alt="Drawing" width="40%" align="left">


### 3.3.1 (1) データ読み込み

In [ ]:
# ガン疾患データセットのロード

# ライブラリのimport
from sklearn.datasets import load_breast_cancer

# データのロード
cancer = load_breast_cancer()

# データの注釈を読む
print(cancer.DESCR)

In [ ]:
# データフレームへの取り込み

columns = [
    '半径_平均', 'きめ_平均', '周長_平均', '面積_平均', 
    '平滑度_平均','コンパクト度_平均', '凹面_平均',
    '凹点_平均', '対称性_平均', 'フラクタル度_平均',
    '半径_標準誤差', 'きめ_標準誤差', '周長_標準誤差',
    '面積_標準誤差', '平滑度_標準誤差',
    'コンパクト度_標準誤差', '凹面_標準誤差', '凹点_標準誤差',
    '対称性_標準誤差', 'フラクタル度_標準誤差',
    '半径_最大', 'きめ_最大', '周長_最大', '面積_最大', 
    '平滑度_最大','コンパクト度_最大', '凹面_最大', '凹点_最大', 
    '対称性_最大', 'フラクタル度_最大'
]

# ロードしたデータのデータフレームへの取り込み
df = pd.DataFrame(cancer.data, columns=columns)

# 正解データの取得
y = pd.Series(cancer.target)

### 3.3.2 (2) データ確認

In [ ]:
# 入力データの表示

# 入力データの先頭20行目から24行目までの表示
display(df[20:25])

In [ ]:
# 正解データの表示

# 正解データの先頭20行目から24行目の表示
print(y[20:25])

In [ ]:
# データの統計情報確認

# 入力データの行数、列数の確認
print(df.shape)
print()

# 正解データの1と0の個数確認
print(y.value_counts())

In [ ]:
# 散布図描画の準備
# データを正解データ=0のグループと正解データ=1のグループに分割する

# 正解データ = 0 (悪性)のデータ抽出
df0 = df[y==0]

# 正解データ = 1(良性)のデータ抽出
df1 = df[y==1]

display(df0.head())
display(df1.head())

In [ ]:
# 散布図表示

# グラフのサイズ設定
plt.figure(figsize=(6,6))

# 目的変数が0のデータを散布図表示
plt.scatter(df0['半径_平均'], df0['きめ_平均'], marker='x',
    c='b', label='悪性')

# 目的変数が1のデータを散布図表示
plt.scatter(df1['半径_平均'], df1['きめ_平均'], marker='s',
    c='k', label='良性')

# 格子表示
plt.grid()

# ラベル表示
plt.xlabel('半径_平均')
plt.ylabel('きめ_平均')

# 凡例表示
plt.legend()

# グラフ表示
plt.show()

### 3.3.3 (3) データ前処理

In [ ]:
# 入力データを2項目だけに絞り込む

input_columns = ['半径_平均', 'きめ_平均']
x = df[input_columns]
display(x.head())

### 3.3.4 (4) データ分割

In [ ]:
# 訓練用データと検証用データの分割

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
    train_size=0.7, test_size=0.3, random_state=random_seed)

In [ ]:
# 分割結果の確認 (要素数)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# 分割結果の確認 (データの内容)

display(x_train.head())
display(x_test.head())
display(y_train.head())
display(y_test.head())

### 3.3.5 (5) アルゴリズム選択

In [ ]:
# アルゴリズム選定

from sklearn.linear_model import LogisticRegression
algorithm = LogisticRegression(random_state=random_seed)

### 3.3.6 (6) 学習

In [ ]:
# 学習

algorithm.fit(x_train, y_train)
print(algorithm)

### 3.3.7 (7) 予測

In [ ]:
# 予測

# predict関数の呼出し
y_pred = algorithm.predict(x_test)

# 結果の確認
print(y_pred)

### 3.3.8 (8) 評価

In [ ]:
# 正解データと予測結果の比較

# 正解データ　先頭から10個
# y_testはDataFrameなので、valuesによりNumPyに変換しておく
y_test10 = y_test[:10].values
print(y_test10)

# 予測結果　先頭から20個
y_pred10 = y_pred[:10]
print(y_pred10)

In [ ]:
# 正解数のカウント

# 正解データ = 予測結果　
w1 = (y_test10 == y_pred10)
print(w1)

# 正解データの数
w2 = w1.sum()
print(w2)

In [ ]:
# 精度の計算

# 正解数の計算
w = (y_test.values == y_pred)
correct = w.sum()

# 検証データ全体数の計算
N = len(w)

# 精度 = (正解数) / (検証データ全体数)
score = correct / N

# 結果表示
print(f'精度: {score:.04f}')

In [ ]:
# score関数の利用

# 実は精度は score関数で簡単に計算できる
score = algorithm.score(x_test, y_test)
print(f'score: {score:.04f}')

### 3.3.9 (9) チューニング

In [ ]:
# モデルの精度を上げる

# オリジナルの30項目の入力データを使って、訓練データ、検証データを作り直す
x2_train, x2_test, y_train, y_test = train_test_split(df, y, 
    train_size=0.7, test_size=0.3, random_state=random_seed)

# ロジスティック回帰モデルのインスタンスを新たに作り直す
algorithm2 = LogisticRegression(random_state=random_seed)

# 訓練データで学習
algorithm2.fit(x2_train, y_train)

# 検証データで精度を確認
score2 = algorithm2.score(x2_test, y_test)
print(f'score: {score2:.04f}')

## (補足) 決定境界の表示
以下のセルでは、決定境界を表示するためのコードと、なぜそのような実装になるかの簡単な解説を記載しています。  
かなり高度は内容ですので、あまりPythonに詳しくない読者は、飛ばしてもらって構いません。  
Pytho実装に関心のある読者は参考とするようにしてください。

### ロジスティック回帰の内部構造

ロジスティック回帰モデルとは、  
(1) 入力変数を一次関数にかけて実数値を導出  
(2) (1)で得られた一次関数値をシグモイド関数と呼ばれる関数にかけて確率値を算出  
(3) (2)の結果が0.5より大きいか小さいかで予測結果が1か0かを判断
という処理を内部的に行っています。  

(1)で使われる一次関数の傾きと切片はそれぞれ変数coef_とintercept_で取得可能です。  
以下のコードではこの性質を使って、内部変数値を取得しています。

なお、このモデルでは多値分類用に、複数の内部変数を保持できるようになっています。
そのため、配列が2次元になっていますが、今回利用するのは2値分類なので、最初の要素([0])の値のみ利用します。



### 内部変数値の取得

In [ ]:
# モデルの内部変数(切片と係数)の取得

# x1とx2の係数
w1 = algorithm.coef_[0][0]
w2 = algorithm.coef_[0][1]

# 切片の値
w0 = algorithm.intercept_[0]

# 値の確認
print(f'w0 = {w0:.4f}  w1 = {w1:.4f}  w2 = {w2:.4f}')

### boundary関数の定義

ここで得られた w0, w1, w2の値を用いると、散布図上に決定境界を示すための関数boundaryを決めることができます。  
具体的な関数と、それを導出するための式を以下のセルで示しました。

In [ ]:
# 決定境界計算用関数

# 決定境界計算用関数
# 0 = w0 + w1 * x + w2 * y をyについて解くと以下の式になる
# y = -(w0 + w1 * x)/ w2

def boundary(algorithm, x):
    w1 = algorithm.coef_[0][0]
    w2 = algorithm.coef_[0][1]
    w0 = algorithm.intercept_[0]
    y = -(w0 + w1 * x)/w2
    return y

### 決定境界の端点値の計算

次に示すコードは、上で定義したboundary関数を利用して、決定境界を構成する端点の座標を求めるためのものです。  
あわせて、元のデータフレームからyの最小値と最大値を求め、グラフの見栄えをよくします 。

In [ ]:
# 決定境界の端点値計算

# 決定境界の端点のx座標
x_range = np.array((df['半径_平均'].min(), df['半径_平均'].max()))

# 決定境界の端点のy座標
y_range = boundary(algorithm, x_range)

# yの上限、下限は散布図の点を元に決める
y_lim = (df['きめ_平均'].min(), df['きめ_平均'].max())

# 結果確認
print('端点のx座標: ', x_range)
print('端点のy座標: ', y_range)
print('グラフのy領域: ', y_lim)

### 散布図と決定境界の表示

これですべての準備は整いました。  
次に示すコードでは、コード3-8で表示した学習データの散布図と、今求めた決定境界を重ね描きしています。

In [ ]:
# 散布図と決定境界の表示

# グラフのサイズ設定
plt.figure(figsize=(6,6))

# 目的変数が0のデータを散布図表示
plt.scatter(df0['半径_平均'], df0['きめ_平均'], marker='x', c='b', label='悪性')

# 目的変数が1のデータを散布図表示
plt.scatter(df1['半径_平均'], df1['きめ_平均'], marker='s', c='k', label='良性')

# 決定境界
plt.plot(x_range, y_range, c='b', label='決定境界')

# 範囲指定
plt.ylim(y_lim)

# ラベル表示
plt.xlabel('半径_平均')
plt.ylabel('きめ_平均')

# 凡例表示
plt.legend()

# 方眼表示
plt.grid()

# グラフ表示
plt.show()